# Day 01 - bigram approach

In [1]:
NAMES_PATH = "/Users/mikolaj/Developer/powielacz/data/names.txt"

In [2]:
words = open(NAMES_PATH, "r").read().splitlines()
print(words[:10])
print(len(words))

['egon', 'dymitr', 'rudolf', 'ingrid', 'berta', 'aleksander', 'adam', 'gerhard', 'balbina', 'teresa']
600


In [3]:
min([len(word) for word in words])

3

In [4]:
max([len(word) for word in words])

11

In [10]:
b = {}
for w in words:
    chs = ["<S>"] + list(w) + ["<E>"]
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        # print(ch1, ch2)

In [19]:
sorted(b.items(),key=lambda kv: kv[1], reverse=True)

[(('a', '<E>'), 300),
 (('n', 'a'), 112),
 (('a', 'n'), 90),
 (('a', 'r'), 74),
 (('e', 'r'), 69),
 (('l', 'i'), 66),
 (('<S>', 'a'), 61),
 (('i', 'a'), 61),
 (('e', 'l'), 60),
 (('i', 'n'), 58),
 (('t', 'a'), 53),
 (('n', '<E>'), 52),
 (('a', 'w'), 52),
 (('ł', 'a'), 52),
 (('o', 'n'), 50),
 (('s', 'z'), 50),
 (('m', 'i'), 49),
 (('<S>', 'm'), 48),
 (('n', 'i'), 47),
 (('m', 'a'), 46),
 (('<S>', 'w'), 46),
 (('<S>', 'e'), 44),
 (('s', 'ł'), 44),
 (('<S>', 'b'), 43),
 (('l', 'a'), 43),
 (('l', 'e'), 42),
 (('e', 'n'), 42),
 (('u', 's'), 42),
 (('z', '<E>'), 42),
 (('r', 'a'), 42),
 (('a', 'l'), 41),
 (('d', 'a'), 41),
 (('<S>', 'l'), 41),
 (('<S>', 'k'), 39),
 (('r', 'y'), 37),
 (('o', 'l'), 36),
 (('<S>', 'j'), 36),
 (('w', 'a'), 36),
 (('w', 'i'), 36),
 (('d', '<E>'), 34),
 (('<S>', 'r'), 33),
 (('s', 't'), 33),
 (('k', 'a'), 33),
 (('i', 'e'), 32),
 (('r', 'o'), 31),
 (('j', 'a'), 30),
 (('w', '<E>'), 28),
 (('d', 'o'), 26),
 (('<S>', 's'), 26),
 (('<S>', 'd'), 25),
 (('r', '<E>'), 

In [21]:
import torch

/Users/mikolaj/Developer/powielacz/.venv/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:283: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [24]:
a = torch.zeros((1,1))
a

tensor([[0.]])